# Advanced Brian 2

In [ ]:
%matplotlib inline
from brian2 import *
import os

def tabula_rasa():
    device.reinit()
    start_scope()
    set_device('runtime')

## Custom functions

In [ ]:
tabula_rasa()
prefs.codegen.target = 'numpy'
#prefs.codegen.target = 'weave'

@check_units(t=second, result=1)
def f(t):
    return sin(2*pi*10*Hz*t)

G = NeuronGroup(1, 'dv/dt=f(t)/(10*ms):1')
M = StateMonitor(G, 'v', record=True)
run(100*ms)
plot(M.t, M.v[0])

In [ ]:
tabula_rasa()
prefs.codegen.target = 'auto'
BrianLogger.log_level_info()

@implementation('cpp', '''
#include<math.h>
double f(double t)
{
    return sin(2*M_PI*10*t);
}
''')
@check_units(t=second, result=1)
def f(t):
    return sin(2*pi*10*Hz*t)

G = NeuronGroup(1, 'dv/dt=f(t)/(10*ms):1')
M = StateMonitor(G, 'v', record=True)
run(100*ms)
plot(M.t, M.v[0])

## Standalone code generation

In [ ]:
tabula_rasa()
set_device('cpp_standalone_simple')

@implementation('cpp', '''
#include<math.h>
double f(double t)
{
    return sin(2*M_PI*10*t);
}
''')
@check_units(t=second, result=1)
def f(t):
    return sin(2*pi*10*Hz*t)

G = NeuronGroup(1, 'dv/dt=f(t)/(10*ms):1')
M = StateMonitor(G, 'v', record=True)
run(100*ms)
plot(M.t, M.v[0])

In [ ]:
tabula_rasa()
set_device('cpp_standalone')

@implementation('cpp', '''
#include<math.h>
double f(double t)
{
    return sin(2*M_PI*10*t);
}
''')
@check_units(t=second, result=1)
def f(t):
    return sin(2*pi*10*Hz*t)

G = NeuronGroup(1, 'dv/dt=f(t)/(10*ms):1', name='G1')
M = StateMonitor(G, 'v', record=True)
run(100*ms)
device.build(directory='output', compile=True, run=True)
plot(M.t, M.v[0])

In [ ]:
ls output

In [ ]:
print open('output/code_objects/G1_stateupdater_codeobject.cpp', 'r').read()

To use GeNN:

In [ ]:
import brian2genn
set_device('genn')

Unfortunately I don't have an NVIDIA card on this laptop, so I can't show you.

But it works.

Honest.

## Limitations of standalone and workarounds

Python code is not translated.

In [ ]:
tabula_rasa()
set_device('cpp_standalone')

G = NeuronGroup(10, 'v:1', name='G2')
G.v = rand(len(G))
run(1*ms)
device.build(directory='output', compile=True, run=True)
plot(G.v)

In [ ]:
os.system(r'cd output && main && cd ..')
plot(G.v)

Brian sees ``neurons.v = an_array_of_values``, and this is stored in a file when you run the Python code:

In [ ]:
plot(fromfile('output/static_arrays/_static_array__array_G2_v'))

Solution: use string-based initialization.

In [ ]:
tabula_rasa()
set_device('cpp_standalone')

G = NeuronGroup(10, 'v:1')
G.v = 'rand()'
run(1*ms)
device.build(directory='output', compile=True, run=True)
plot(G.v)

In [ ]:
os.system(r'cd output && main && cd ..')
plot(G.v)

## More flexibility

You can insert C++ code directly into the generated code, e.g.:

In [ ]:
tabula_rasa()
set_device('cpp_standalone')

#device.insert_code('main', 'srand(3402343);')
G = NeuronGroup(1, 'v:1')
G.v = 'rand()'
device.insert_code('main', 'cout << "Hello!" << endl;')
run(1*ms)
device.build(directory='output', compile=True)

In [ ]:
# IPython hides stdout by default, so we have to use subprocess to see it.
import subprocess
os.chdir('output')
print subprocess.check_output(["./main"])  
# print subprocess.check_output(["main"])  # use this line on Windows instead
os.chdir('..')
print G.v

## Interfacing with your own code

Brian has some features to help ease the process of integrating generated code with a fixed library.

Talk to us if you want to do this because we're still working on what would be most useful.